Code pour projet crypto

Import des librairies

**Date limite :** Mercredi 27 mars 2024

**Groupe TD DIA 6 :**

Henri Serano, Sara Thibierge, Eloi Seidlitz

In [3]:
import hashlib
import random 
import gmpy2 as gm
import math
import time
import base64
import os, binascii
from backports.pbkdf2 import pbkdf2_hmac
import pyaes, pbkdf2, secrets
import nbdime


In [33]:
!nbdime config-git --enable

## Exercice 1

1. Trouver une chaîne de caractères (contenant vos noms et prénoms) dont le hash SHA256 se termine par le plus de zéros possible (en hexadécimal).

2. Mesurer le temps moyen pour obtenir n et n+1 zéros en fin de chaîne (n = 5) et calculer le rapport Tn+1/Tn.

### Partie 1

Nous allons faire une fonction permettant de trouver une combinaison du chiffrage de la phrase avec l'algorithme sha256 qui finit avec un nombre finit de 0

In [8]:
def trouver_chaine(base, zeros_cibles):
        compteur = 0
        while True:
            chaine_test = base + str(compteur)
            hash_resultat = sha256_hash(chaine_test)
            if hash_resultat.endswith('0' * zeros_cibles):
                return chaine_test, compteur, hash_resultat
            compteur += 1

def sha256_hash(chaine):
        return hashlib.sha256(chaine.encode()).hexdigest()

In [46]:
def trouver_hash_avec_zeros(texte, nombre_zeros):
    nonce = 0
    zeros_cibles = '0' * nombre_zeros
    while True:
        texte_nonce = f"{texte}{nonce}"
        hash_result = hashlib.sha256(texte_nonce.encode()).hexdigest()
        if hash_result.endswith(zeros_cibles):
            return texte_nonce, nonce, hash_result
        nonce += 1

texte = "Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6a"
nombre_zeros = 4

texte_nonce, nonce_trouve, hash_resultat = trouver_hash_avec_zeros(texte, nombre_zeros)
print(f"chaine_trouvée: {texte_nonce}")
print(f"Nonce trouvé: {nonce_trouve}")
print(f"Hash correspondant: {hash_resultat}")

chaine_trouvée: Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6a162278
Nonce trouvé: 162278
Hash correspondant: 9a08ff6abdfc740cc320f0feced49520aea2a3999c15a52b1a42ffa49f300000


Cette fonction permet d'executer la fonction et de trouver les différentes statistique en lien avec ce cryptage

### Partie 2

In [50]:
def trouver_chaine_et_stats(base, zeros_cibles, iterations_stat=30):

    temps_n = []
    temps_n_plus_1 = []
    print("|                Chaine de caractère                |      Temps (n)    \t | nonce pour n zéros\t|     Temps (n+1)\t| nonce pour n+1 zéros\t|")
    print("|\t\t\t\t\t\t    |\t\t\t\t | \t\t\t|\t\t\t| \t\t\t|")

    for i in range(iterations_stat):
        start = time.time()
        texte_nonce_n, nonce_trouve_n, hash_resultat_n = trouver_hash_avec_zeros(base+chr(97+i), zeros_cibles)
        stop = time.time()
        temps_n.append(stop - start)

        start = time.time()
        texte_nonce_n_plus_1, nonce_trouve_n_plus_1, hash_resultat_n_plus_1 = trouver_hash_avec_zeros(base+chr(97+i), zeros_cibles + 1)
        stop = time.time()
        temps_n_plus_1.append(stop - start)
        print(f"| {base+chr(97+i)} | {temps_n[-1]}    \t | {nonce_trouve_n}   \t\t| {temps_n_plus_1[-1]}\t| {nonce_trouve_n_plus_1}   \t\t| ")

    temps_moyen_n = sum(temps_n) / len(temps_n)
    temps_moyen_n_plus_1 = sum(temps_n_plus_1) / len(temps_n_plus_1)

    rapport_temps = temps_moyen_n_plus_1 / temps_moyen_n

    return {
        "nombre_n_de_zeros": zeros_cibles,
        "temps_moyen_n": temps_moyen_n,
        "temps_moyen_n+1": temps_moyen_n_plus_1,
        "rapport_temps": rapport_temps
    }


base_chaine = "Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6"
zeros_cibles = 4 # Nombre de zéros à la fin du hash
resultats = trouver_chaine_et_stats(base_chaine, zeros_cibles)
resultats


|                Chaine de caractère                |      Temps (n)    	 | nonce pour n zéros	|     Temps (n+1)	| nonce pour n+1 zéros	|
|						    |				 | 			|			| 			|
| Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6a | 0.402634859085083    	 | 162278   		| 0.36391687393188477	| 162278   		| 
| Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6b | 0.7915651798248291    	 | 350066   		| 7.1882240772247314	| 2722191   		| 
| Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6c | 0.185302734375    	 | 86137   		| 1.9304389953613281	| 827772   		| 
| Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6d | 0.13721013069152832    	 | 61749   		| 0.42972397804260254	| 193292   		| 
| Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6e | 0.010272026062011719    	 | 4623   		| 1.476850986480713	| 664677   		| 


{'nombre_n_de_zeros': 4,
 'temps_moyen_n': 0.30539698600769044,
 'temps_moyen_n+1': 2.277830982208252,
 'rapport_temps': 7.458590249973495}

## Exercice 2

1. Creation of a function that will create a number with a chosen lenth (ex: 6 to 9) and ramdom digits using only the ones given (ex: 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [74]:
def generate_selected_digit_number(length,selected_digit):
    return int(''.join(secrets.choice(selected_digit) for _ in range(length)))

2. Creation of the password and the initial vector

In [75]:
non_zero = "123456789"
password = generate_selected_digit_number(random.randint(6, 9),non_zero)
iv = generate_selected_digit_number(random.randint(6, 9),non_zero)

3. get the integer N

In [76]:
N = int(str(password) + "0000" + str(iv))

In [113]:
# print(f"Password:\t{password}")
# print(f"IV:\t\t{iv}")
# print(f"N:\t\t{N}")

4. Now lets encrypt our text with AES 256 CTR PBKDF2 using our keys

Keep in mind that we remplaced our N and iv by there name to avoid you to see there value

In [116]:
!openssl enc -aes-256-ctr -pbkdf2 -in text_to_encrypt.txt -out ES_HS_ST.enc -pass pass:N -iv iv
# !openssl enc -d -aes-256-ctr -pbkdf2 -in ES_HS_ST.enc -out text_to_encrypt2.txt -pass pass:N -iv iv

hex string is too short, padding with zero bytes to length
non-hex digit
invalid hex iv value


In [108]:
# Le nombre premier p :                                      
p = 7946851324679854613245823 
# Le « générateur » d’un groupe d’ordre élevé : 
g = 5 
# Clé publique A de Herbert Groscot:                                         
A = 7579501795988122393422986

b = random.SystemRandom().randint(2, p-2)

# Calcul de B et C pour ElGamal
B = pow(g, b, p)
K = pow(A, b, p)  
C = (K * N) % p  

print(f"Clé publique B de Eloi, Henri et Sara: \t{B}")
print(f"Chiffrement C de lentier N: \t\t{C}")

Clé publique B de Eloi, Henri et Sara: 	7928524311388737963568910
Chiffrement C de lentier N: 		6207001348060844298451324


## Exercice 3

* Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).
* Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

#### 1. Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).

In [24]:
# Fonction pour calculer l'inverse modulo
def inverse_modulo(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

# Fonction pour calculer la signature
def sign_elgamal(m, x, p, g):
    # Sélection d'un nombre aléatoire k
    k = random.randint(2, p - 2)
    while pow(k, -1, p - 1) == 0:  # Assure que k est premier avec p-1
        k = random.randint(2, p - 2)
    r = pow(g, k, p)
    k_inv = inverse_modulo(k, p - 1)
    s = (m - x * r) * k_inv % (p - 1)
    return r, s

# Fonction pour vérifier la signature
def verify_elgamal(m, r, s, y, p, g):
    v1 = pow(g, m, p)
    v2 = (pow(y, r, p) * pow(r, s, p)) % p
    return v1 == v2

# Paramètres ElGamal
p = 257
g = 3
x = 97

# Clé publique
y = pow(g, x, p)

# Message à signer
m = 123

# Signature
r, s = sign_elgamal(m, x, p, g)

# Vérification de la signature
valid = verify_elgamal(m, r, s, y, p, g)

print("Signature valide :", valid)


Signature valide : True


#### 2. Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

Supposons que nous ayons un fichier texte appelé "format.txt" contenant le message à signer.



**a) Génération des clés RSA :**
Tout d'abord, nous devons générer une paire de clés RSA privée/publique à l'aide d'OpenSSL.

In [25]:
!openssl genpkey -algorithm RSA -out private_key.pem
!openssl rsa -pubout -in private_key.pem -out public_key.pem

....+...+..+++++++++++++++++++++++++++++++++++++++*.+.............+.........+.....+..........+..+...+.......+.....+++++++++++++++++++++++++++++++++++++++*..+.........+......+..........+.....+............+...+...................+...+......+...........+...+............+.+........+......+......+.......+..+.+...+.........+...+..+...+....+.........+.....+....+...+..+.+..+...+......+.+.................+............+...+.......+...+...+..............+....+..+.......+..+....++++++
........+.......+..............+......+++++++++++++++++++++++++++++++++++++++*..+..................+....+...+...+...+.....+.+......+.........+......+.....+.........+.+..+.............+........+.......+..+...+.............+.....+....+......+......+.....+.+..+++++++++++++++++++++++++++++++++++++++*........++++++


writing RSA key


Ces commandes génèrent une clé privée private_key.pem et une clé publique correspondante `public_key.pem`.



**b) Signature du message :**

Ensuite, nous signons le message avec notre clé privée.

In [26]:
!openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt

Cela crée une signature du message dans le fichier signature.bin.


**c) Vérification de la signature :**

Pour vérifier la signature avec la clé publique correspondante, nous utilisons :

In [27]:
!openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt

Verified OK


Si la vérification réussit, OpenSSL affichera "Verified OK".
